## Recap курса "Введение в машинное обучение"

## Семинар #10

### Екатерина Кондратьева

ekaterina.kondrateva@skoltech.ru

  Программа курса:
  
   1. Обучение с учителем: Линейная и логистическая регрессия. Ядра.
    
    Фолдер: https://github.com/kondratevakate/machine-learning-with-love/tree/master/02_lr
    
 
    
   2. Обучение с учителем: Регуляризация в линейных моделях. Метод Ближайших Соседей (KNN)
    
    Фолдер: https://github.com/kondratevakate/machine-learning-with-love/blob/master/03_knn
    

    
   3. Обучение с учителем: Метод опорных векторов (SVM) для задач классификации и регрессии. Kernel SVM
    
    Фолдер: https://github.com/kondratevakate/machine-learning-with-love/tree/master/04_svc
     

    
   4. Обучение с учителем: Деревья решений (Decision Trees). Случайный лес (Random Forest).
    
    Фолдер: https://github.com/kondratevakate/machine-learning-with-love/tree/master/05_rfc
    

    
   5. Оценка качества алгоритмов машинного обучения. Кросс-валидация. Поиск аномалий и артефактов в выборке. 
    
    Фолдер: https://github.com/kondratevakate/machine-learning-with-love/tree/master/06_model_validaton
    

    
   6. Обучение без учителя: кластеризация. Снижение размерности данных PCA.
    
    Фолдер: https://github.com/kondratevakate/machine-learning-with-love/tree/master/07_clustering_pca
    

    
   7. Отбор и генерация признаков (Feature Engineering). Поиск и оптимизация модели (Grid Search).
    
    Фолдер: https://github.com/kondratevakate/machine-learning-with-love/tree/master/08_feature_engineering
    

    
   8. Стекинг, вотинг. Градиентный бустинг. Пакеты XGBoost/Catboost/LightGBM  
    
    Фолдер: https://github.com/kondratevakate/machine-learning-with-love/tree/master/09_gradient_boosting
    
    

### Построим пайплан для датасета HousePricing:

Данные - https://www.kaggle.com/lider123/housepricing

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import skew
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import mean_squared_error
import seaborn as sns

In [ ]:
train_data = pd.read_csv("housepricing/train.csv")
test_data = pd.read_csv("housepricing/test.csv")

In [ ]:
full_data = pd.concat([train_data,test_data])

In [ ]:
full_data.info()

In [ ]:
categorial_features = [prizn for prizn in full_data.columns if full_data[prizn].dtype.name == 'object']
numeric_features = [prizn for prizn in full_data.columns if full_data[prizn].dtype.name != 'object']
print(categorial_features, numeric_features, sep = '\n')

In [ ]:
# заполним пропуски числовых данных применяя свои знания предметной области
full_data["SaleType"].fillna(full_data["SaleType"].mode()[0], inplace=True)
full_data["Electrical"].fillna(full_data["Electrical"].mode()[0], inplace=True)
full_data["MSZoning"].fillna(full_data["MSZoning"].mode()[0], inplace=True)
full_data["GarageArea"].fillna(full_data["GarageArea"].median(), inplace=True)
full_data["CentralAir"] = full_data["CentralAir"].map({"Y": 1, "N": 0})
full_data["LotFrontage"].fillna(full_data["LotFrontage"].median(), inplace=True)
full_data["Alley"].fillna("None", inplace=True)
full_data["MasVnrType"].fillna("None", inplace=True)
full_data["MasVnrArea"].fillna(0, inplace=True)

In [ ]:
#заполним пропуски числовых данных подряд медианой
for i in numeric_features:
    full_data[i].fillna(full_data[i].median(), inplace=True)

In [ ]:
# кодируем переменные типа объекта
full_data = pd.get_dummies(full_data, columns=categorial_features)

X_train, X_test = full_data.iloc[:len(train_data), :], full_data.iloc[len(train_data):, :]

X_train.head()

In [ ]:
X_train.shape

#### Снижение размерности?
#### Какая задача - регрессия или классификация?

In [ ]:
# построим самый простой регрессор
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score, StratifiedShuffleSplit, StratifiedKFold

clf = LinearRegression()

#### Регуляризация?
#### Оценка модели? 

In [ ]:
y_train = train_data["SalePrice"]
X_train.drop("SalePrice",axis=1, inplace=True)

In [ ]:
y_train.shape

In [ ]:
X = X_train
y = y_train
skf = StratifiedKFold(n_splits=5, random_state=42)
skf.get_n_splits(X, y)

scores=[]

for train_index, test_index in skf.split(X, y):
    #print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    clf.fit(X_train, y_train)
    print(clf.score(X_test, y_test))
    scores.append(clf.score(X_test, y_test))

In [ ]:
from sklearn.preprocessing import StandardScaler

sclr= StandardScaler()
X_cols= X_train.columns
X = pd.DataFrame(sclr.fit_transform(X_train), columns = X_cols)


y = y_train
skf = StratifiedKFold(n_splits=5, random_state=42)
skf.get_n_splits(X, y)

scores=[]

for train_index, test_index in skf.split(X, y):
    #print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    clf.fit(X_train, y_train)
    print(clf.score(X_test, y_test))
    scores.append(clf.score(X_test, y_test))

In [ ]:
cross_val_score(lrr, X_train, y_train, cv=5)

In [ ]:
X_train_short= X_train[['Id', 'MSSubClass', 'LotArea', 'OverallQual',\
      'OverallCond', 'YearBuilt', 'YearRemodAdd',\
      'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF',\
     'LowQualFinSF', 'GrLivArea', 'BsmtFullBath',\
     'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr',\
     'TotRmsAbvGrd', 'Fireplaces', 'GarageCars',\
     'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch',\
     'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold']]

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

params = {
    "n_estimators": [10, 50],
    "max_features": ["auto", "sqrt"],
    "max_depth": [5,10]
}
gs = GridSearchCV(estimator=RandomForestRegressor(), param_grid=params, cv=5, scoring="neg_mean_squared_error")
gs.fit(X_train, y_train)

gs.best_params_

In [ ]:
model = RandomForestRegressor(**gs.best_params_)
cross_val_score(model, X_train, y_train, cv=5)

In [ ]:
X = X_train
y = y_train
skf = StratifiedKFold(n_splits=5, random_state=42)
skf.get_n_splits(X, y)

scores=[]

for train_index, test_index in skf.split(X, y):
    #print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    model.fit(X_train, y_train)
    print(model.score(X_test, y_test))
    scores.append(model.score(X_test, y_test))

In [ ]:
X = X_train
y = y_train
skf = StratifiedKFold(n_splits=5, random_state=42)
skf.get_n_splits(X, y)

scores=[]

for train_index, test_index in skf.split(X, y):
    #print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    clf.fit(X_train, y_train)
    print(clf.score(X_test, y_test))
    scores.append(clf.score(X_test, y_test))

## Titanic dataset 

In [ ]:
train_df = pd.read_csv('titanic/train.csv')
test_df = pd.read_csv('titanic/test.csv')

In [ ]:
train_df["Male"] = train_df.Sex.map({"male": 1, "female": 0}).astype(int) # альтернатива OneHotEncoding
test_df["Male"] = test_df.Sex.map({"male": 1, "female": 0}).astype(int)

train_df = train_df.drop(["PassengerId", "Ticket", "Cabin", "Name", "Sex", "SibSp", "Parch", "Embarked", "Fare", 'Age'], axis=1)
test_df = test_df.drop(["Ticket", "Cabin", "Name", "Sex", "SibSp", "Parch", "Embarked", "Fare",'Age'], axis=1)

X = train_df.drop("Survived", axis=1).fillna(np.nan)
y = train_df.Survived
X_test = test_df.drop("PassengerId", axis=1).fillna(np.nan)


X.tail()

In [ ]:
from sklearn.linear_model import LogisticRegression
clf= LogisticRegression()

In [ ]:
skf = StratifiedKFold(n_splits=10, random_state=42)
skf.get_n_splits(X, y)

scores=[]

for train_index, test_index in skf.split(X, y):
    #print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    clf.fit(X_train, y_train)
    print(clf.score(X_test, y_test))
    scores.append(clf.score(X_test, y_test))

In [ ]:
clf = Pipeline([('selectk', SelectKBest(k=6)), 
                     ('scaler', StandardScaler()),
                ('decision_tree',  RandomForestClassifier(n_estimators=25, max_depth=3))])

cross_val_score(estimator=clf, X=X_train, y=y_train, cv=5).mean()

### Метрики: оценки качества

### Практикум по *.Git системам:

!['git'](https://i1.wp.com/lanziani.com/slides/gitflow/images/gitflow_1.png?zoom=2)

1. Установка: https://git-scm.com/downloads
2. GUI: https://desktop.github.com/
3. Туториал: https://learngitbranching.js.org/